In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../netflix_titles.csv')
#df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

8807

In [3]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [4]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

/home/rho/Documents/Coursera/learn-RAG/repo/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [6]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="netflix_titles",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/tmp/ipykernel_1443695/2012531779.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [ ]:
# vectorize!
qdrant.upload_points(
    collection_name="netflix_titles",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["description"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [ ]:
user_prompt = "Suggest me movie about an alien invasion"

In [ ]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="netflix_titles",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

In [ ]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [ ]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)

In [ ]:
client.models.list()

In [ ]:
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a netflix content specialist. Your top priority is to help guide users into selecting movies or shows."},
        {"role": "user", "content": "Suggest me a movie about an alien invasion"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)